In [56]:
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet
from sklearn.metrics import r2_score

# Load training and test data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Ensure all dates are in datetime format
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

# Get the top 10 stations by total rides in training data
top_stations = train.groupby('start_station_name')['total_rides'].sum().nlargest(10).index

# Store results and models
results = []
station_models = {}

for station_name in top_stations:
    print(f"Processing: {station_name}")

    # Filter data for the current station
    station_train_data = train[train.start_station_name == station_name].drop(columns=['start_station_name'])
    station_test_data = test[test.start_station_name == station_name].drop(columns=['start_station_name'])

    # Rename columns to match NeuralProphet requirements
    station_train_data.rename(columns={'date': 'ds', 'total_rides': 'y'}, inplace=True)
    station_test_data.rename(columns={'date': 'ds', 'total_rides': 'y'}, inplace=True)

    # Filter only the required columns
    station_train_data = station_train_data[['ds', 'y', 'temp_min_c']]
    station_test_data = station_test_data[['ds', 'y', 'temp_min_c']]

    # Ensure no negative or zero values in y (total rides)
    station_train_data = station_train_data[station_train_data['y'] > 0]
    station_test_data = station_test_data[station_test_data['y'] > 0]

    # Initialize NeuralProphet model
    model = NeuralProphet(
        epochs=100,
        yearly_seasonality='auto',
        weekly_seasonality='auto',
        loss_func="Huber",  
        normalize='soft'  
    )

    # Add temperature as a future regressor
    model.add_future_regressor('temp_min_c')

    # Train model with early stopping
    model.fit(station_train_data, freq='D', early_stopping=True)

    # Store model in dictionary
    station_models[station_name] = model

    # Prepare test data for forecasting
    future = station_test_data[['ds', 'temp_min_c']].copy()
    future['y'] = station_test_data['y']

    # Predict future values
    forecast = model.predict(future)

    # Ensure predictions are non-negative
    forecast['yhat1'] = forecast['yhat1'].clip(lower=0)

    # Merge forecast with actual values
    result = forecast[['ds', 'yhat1']].merge(station_test_data[['ds', 'y']], on='ds', how='left')

    # Compute R² score
    y_actual = result['y'].dropna()
    y_predicted = result.loc[y_actual.index, 'yhat1']
    r2 = r2_score(y_actual, y_predicted)

    # Store result
    results.append([station_name, r2])

# Convert results into DataFrame for display
r2_df = pd.DataFrame(results, columns=['Station Name', 'R² Score'])

# Instructions for plotting
print("\nTo visualize a station's forecast, run:")
print("model = station_models['Your Station Name']")
print("forecast = model.predict(future)")  # Assuming you redefine future based on test data
print("model.plot(forecast)")


WARNING - (py.warnings._showwarnmsg) - /var/folders/sk/1lq3mdt55q3579qwycd7hv4c0000gn/T/ipykernel_92397/84660843.py:7: DtypeWarning:

Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.


WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 98.958% of the data.
WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/

Processing: Streeter Dr & Grand Ave


Finding best initial lr:   0%|          | 0/229 [00:00<?, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/lib/python3.12/site-packages/lightning_fabric/utilities/cloud_io.py:51: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.




Training: 0it [00:00, ?it/s]

...

Training: 0it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.727% of the data.
WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/

Predicting: 46it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column



To visualize a station's forecast, run:
model = station_models['Your Station Name']
forecast = model.predict(future)
model.plot(forecast)


In [57]:
# Get the trained model for a specific station
station_name = "Clark St & Elm St"  # Replace with any station from top 10
model = station_models[station_name]

# Recreate the correct test dataset (future) for this station
station_test_data = test[test.start_station_name == station_name].drop(columns=['start_station_name'])

# Rename columns to match NeuralProphet requirements
station_test_data.rename(columns={'date': 'ds', 'total_rides': 'y'}, inplace=True)

# Ensure we only keep relevant columns
future = station_test_data[['ds', 'temp_min_c']].copy()
future['y'] = station_test_data['y']  # Include actual values for merging

# Predict using the correct test data
forecast = model.predict(future)

# Plot the forecast
model.plot(forecast)

WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.452% of the data.
WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/

Predicting: 46it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
ERROR - (NP.plotly.plot) - plotly-resampler is not installed. Please install it to use the resampler.
WARNING - (py.warnings._showwarnmsg) - /opt/anaconda3/lib/python3.12/site-packages/neuralprophet/plot_forecast_plotly.py:100: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result


